In [1]:
%pylab inline

import os
from src.CAPTCHA_to_signs import DFS

Populating the interactive namespace from numpy and matplotlib


## Preprocess

W tym miejscu chcemy podzielić surowe CAPTCHA ze znanymi etykietami na poszczególne znaki z również znanymi etykietami. Powstały zbiór wykorzystamy do nauki modelu

In [2]:
root_path = os.getcwd()
img_dir_path = os.path.join(root_path, 'data', 'raw')
invalid_img_dir_path = os.path.join(root_path, 'data', 'invalid')
signs_output_dir_path = os.path.join(root_path, 'data', 'signs')

In [3]:
for sign in '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    sign_dir_path = os.path.join(signs_output_dir_path, sign)
    
    if not os.path.exists(sign_dir_path):
        os.makedirs(sign_dir_path)

In [4]:
from collections import Counter

counter = Counter()

for img_filename in os.listdir(img_dir_path):
    img_path = os.path.join(img_dir_path, img_filename)
    
    label = img_filename.split('.')[0].upper()
    signs = DFS(img_path).dfs_all()
    
    # Przenieś niepoprawne obrazki do folderu invalid
    if len(label) != len(signs):
        invalid_img_path = os.path.join(invalid_img_dir_path, img_filename)
        os.rename(img_path, invalid_img_path)
        continue
        
    for sign, target in zip(signs, label):
        counter[target] += 1
        
        sign_path = os.path.join(signs_output_dir_path, target, str(counter[target]))
        sign.save(sign_path)